In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
model = YOLO("PATH_TO_BEST_PT")
img_path = "PATH_TO_PROCESSED_IMAGE"
orig_img = cv2.imread(img_path)
orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

In [ ]:
def show_img(img, title=""):
    plt.figure(figsize=(6,6))
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")
    plt.show()
show_img(orig_img, "Original Image")

In [ ]:
def run_inference(image, title="Result"):
    results = model(image)
    annotated = results[0].plot()
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    show_img(annotated, title)
    return results[0]

In [ ]:
def add_gaussian_noise(img, mean=0, std=25):
    noise = np.random.normal(mean, std, img.shape).astype(np.float32)
    noisy = np.clip(img + noise, 0, 255).astype(np.uint8)
    return noisy

def add_salt_pepper_noise(img, prob=0.02):
    noisy = img.copy()
    mask = np.random.rand(*img.shape[:2])
    noisy[mask < prob/2] = 0
    noisy[mask > 1 - prob/2] = 255
    return noisy

def change_brightness(img, factor=1.5):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:,:,2] = hsv[:,:,2]*factor
    hsv[:,:,2][hsv[:,:,2]>255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

def apply_blur(img, ksize=15):
    return cv2.GaussianBlur(img, (ksize,ksize), 0)

def rotate_image(img, angle=30):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2,h//2), angle, 1)
    return cv2.warpAffine(img, M, (w,h))

def occlude_image(img, size=80):
    occluded = img.copy()
    h, w = img.shape[:2]
    x, y = np.random.randint(0, w-size), np.random.randint(0, h-size)
    occluded[y:y+size, x:x+size] = 0
    return occluded

In [ ]:
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    if union_area == 0:
        return 0.0
    return inter_area / union_area

def compute_image_iou(preds, gts):
    if len(preds) == 0 or len(gts) == 0:
        return 0.0
    ious = []
    for gt in gts:
        best_iou = 0.0
        for pred in preds:
            iou = compute_iou(pred, gt)
            if iou > best_iou:
                best_iou = iou
        ious.append(best_iou)
    return np.mean(ious)

In [ ]:
perturbations = [
    (add_gaussian_noise, {"std": 30}),
    (add_salt_pepper_noise, {"prob": 0.05}),
    (change_brightness, {"factor": 0.5}),
    (apply_blur, {"ksize": 25}),
    (rotate_image, {"angle": 45}),
    (occlude_image, {"size": 100})
]

for perturb_fn, params in perturbations:
    perturbed_img = perturb_fn(img, **params)
    show_img(perturbed_img, title=f"Perturbed: {perturb_fn.__name__}")
    preds = run_inference(perturbed_img)
    iou = compute_image_iou(preds, gts)
    print(f"IoU for {perturb_fn.__name__}: {iou}")